In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset,DataLoader
import os

from sklearn.model_selection import KFold
from torch import nn
from torchsummary import summary
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


from torch.optim.lr_scheduler import StepLR
import pickle

d:\programfilesssd2\miniconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\programfilesssd2\miniconda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
d:\programfilesssd2\miniconda\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
#  获取数据集最值。
for TYPE in ["EDA"]:
    MAX = 0
    MIN = 1
    PATH = f'D:\\CrossPC_DS\\{TYPE}\\eight\\'
    for file_name in os.listdir(PATH):
        if(os.path.splitext(file_name)[1] == ".pkl"):
            for row in pd.read_pickle(PATH + file_name)[TYPE].values:
                MAX = np.max(row) if np.max(row) > MAX else MAX
                MIN = np.min(row) if np.max(row) < MIN else MIN 
    print(f'{TYPE}数据：max={MAX},min={MIN}')

EDA数据：max=18.19138782,min=0.020522791


In [3]:
# 准备数据集
dataURL = "D:/CrossPC_DS/EDA/eight/EDA300.pkl"
dataX = []
dataY = []
for i in pd.read_pickle(dataURL).values:
    dataX.append(i[4][0:1200])
    dataY.append(i[5])

dataX = torch.unsqueeze(torch.Tensor(np.stack(dataX)),1)  # reshape至(None,1,1200)
dataX = (dataX - MIN)/(MAX - MIN)# 归一化
dataY = torch.LongTensor(dataY)  # 分类长整型

In [4]:
for i in range(len(dataX)):
    pass
    #print(torch.max((dataX)[i]))
print(dataX[110])

tensor([[0.4027, 0.4028, 0.4030,  ..., 0.5772, 0.5775, 0.5777]])


In [5]:
kf = KFold(n_splits=10,shuffle=True,random_state=114514)
rnm   = []
for train_index, valid_index in kf.split(dataX):
    rnm.append([train_index,valid_index])
    pass
# print(dataY[valid_index])
dataset_train = TensorDataset(dataX[train_index],dataY[train_index])
dataset_valid = TensorDataset(dataX[valid_index],dataY[valid_index])

# dataLoader_train = DataLoader(dataset_train, batch_size=32, shuffle=True,num_workers=4,pin_memory=True)
# dataLoader_valid = DataLoader(dataset_valid, batch_size=32, shuffle=False,num_workers=4,pin_memory=True)
dataLoader_train = DataLoader(dataset_train, batch_size=128, shuffle=True,num_workers=4,pin_memory = True,prefetch_factor=16) 
dataLoader_valid = DataLoader(dataset_valid, batch_size=128, shuffle=False,num_workers=4,pin_memory = True,prefetch_factor=16)

In [6]:
# for batch_data, batch_targets in dataLoader_train:
#     print(batch_data)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
ks = 9
strideA = 2
strideB = 1
padding = "valid"
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.cov_unit1 = nn.Sequential(
#             nn.Conv1d(in_channels=1, out_channels=32, kernel_size=ks,stride = 1,padding="same"),
#             nn.Conv1d(in_channels=32, out_channels=32, kernel_size=ks,stride = 1,padding="same"),
#             nn.MaxPool1d(2),
#             nn.ReLU(),
#             nn.Conv1d(in_channels=32, out_channels=64, kernel_size=ks,stride = 1,padding="same"),
#             nn.Conv1d(in_channels=64, out_channels=64, kernel_size=ks,stride = 1,padding="same"),
#             nn.MaxPool1d(2),
#             nn.ReLU(),
#             nn.Conv1d(in_channels=64, out_channels=128, kernel_size=ks,stride = 1,padding="same"),
#             nn.Conv1d(in_channels=128, out_channels=128, kernel_size=ks,stride = 1,padding="same"),
#             nn.MaxPool1d(2),
#             nn.ReLU(),
#             nn.Conv1d(in_channels=128, out_channels=128, kernel_size=ks,stride = 1,padding="same"),
#             nn.Conv1d(in_channels=128, out_channels=128, kernel_size=ks,stride = 1,padding="same"),
#             nn.Conv1d(in_channels=128, out_channels=128, kernel_size=ks,stride = 1,padding="same"),
#             nn.MaxPool1d(2),
#             nn.ReLU(),
#             nn.Flatten(),
            
#             nn.Linear(9600,1200),
#             nn.ReLU(),
#             nn.Linear(1200,100),
#             nn.ReLU(),
#             nn.Linear(100,3),
            
            
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3,stride = 1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
            
            
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3,stride = 1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),
            
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3,stride = 1),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3,stride = 1),
            nn.MaxPool1d(2),
            nn.ReLU(),
            
            nn.BatchNorm1d(64),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3,stride = 1),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3,stride = 1),
            nn.MaxPool1d(2),
            nn.ReLU(),
            
            nn.AdaptiveMaxPool1d(8),
            
            
            
            
            nn.Flatten(),
            
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Dropout(0.85),
            
            nn.Linear(512,100),
            nn.ReLU(),
            nn.Dropout(0.85),
            
            nn.Linear(100,3),
        )

    def forward(self, x):
        
#         print(x.shape)
        x = self.cov_unit1(x)
#         print(x.shape)
#         x = x.view(x.size(0), -1) 
#         print(x.shape)
#         x = self.fc1(x)
#         print(x.size)
        return x

model = Model()
model = model.to(device)
summary(model,(1,1200))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 16, 1198]              64
       BatchNorm1d-2             [-1, 16, 1198]              32
              ReLU-3             [-1, 16, 1198]               0
         MaxPool1d-4              [-1, 16, 599]               0
            Conv1d-5              [-1, 32, 597]           1,568
       BatchNorm1d-6              [-1, 32, 597]              64
              ReLU-7              [-1, 32, 597]               0
         MaxPool1d-8              [-1, 32, 298]               0
            Conv1d-9              [-1, 64, 296]           6,208
           Conv1d-10              [-1, 64, 294]          12,352
        MaxPool1d-11              [-1, 64, 147]               0
             ReLU-12              [-1, 64, 147]               0
      BatchNorm1d-13              [-1, 64, 147]             128
           Conv1d-14             [-1, 1

In [8]:

dataCount = len(dataset_train)
dataCount
# device

2484

In [9]:
class MODEL_LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.LSTM = nn.LSTM(input_size=1200, hidden_size=1200, num_layers=2,dropout=0,batch_first=True)
        
        self.Linear = nn.Sequential(
            nn.Dropout(0.1),
            
            nn.Linear(1200,1440),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(1440,512),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            
            nn.Linear(512,144),
            nn.ReLU(),
            nn.Dropout(0.8),#0.2  num4
            
            nn.Linear(144,3),
        )
        

    def forward(self, x):
        x,_ = self.LSTM(x)
        x = x[:, -1, :]
        x = self.Linear(x)
        return x

In [ ]:
model = MODEL_LSTM()
model = model.to(device)

epochs = 3000
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #0.0008

criterion = torch.nn.CrossEntropyLoss()

scheduler = StepLR(optimizer, step_size=200, gamma=0.97)

lossHistory = []
accHistory = []
trainAccHis = []

# dataLoader_valid = DataLoader(TensorDataset(dataX[rnm[-1][1]],dataY[rnm[-1][1]]), batch_size=64, shuffle=False,num_workers=4)




for epoch in range(0,epochs):
    model.train()
    LOSS = 0
    ACC = 0
    for batch_data, batch_targets in dataLoader_train:
        optimizer.zero_grad()
        outputs = model(batch_data.to(device))
        loss = criterion(outputs, batch_targets.to(device))
        loss.backward()
        
        LOSS += loss.detach().cpu().numpy()
        ACC += (torch.max(outputs,dim=1)[1] == batch_targets.to(device)).sum()
        
        optimizer.step()
        
    scheduler.step()
    
    lossHistory.append(LOSS)
    
    print(f"epoch{epoch+1}: {str(LOSS)[0:7]}",end="\t ")
    cal = (ACC/dataCount).cpu()
    print(f"acc_t[{str(np.array(cal))[0:5]}]",end="\t ")
    trainAccHis.append(cal)
    print("lr["+str(optimizer.param_groups[0]['lr'])+"]",end="\t ")
    
#     for p in model.parameters():
#         print(p.grad.data)
    
    model.eval()
    with torch.no_grad():
        all_outputs = []
        all_targets = []
        for batch_data, batch_targets in dataLoader_valid:
            outputs = model(batch_data.to(device))
            _, predicted = torch.max(outputs.data, 1)
            all_outputs.append(predicted.cpu().numpy())
            all_targets.append(batch_targets.cpu().numpy())

        all_outputs = np.concatenate(all_outputs)
        all_targets = np.concatenate(all_targets)
        accuracy = accuracy_score(all_targets, all_outputs)
        accHistory.append(accuracy)
#         accAverHis.append(np.mean(np.array(accHistory)))
        print(f"acc_val: {str(accuracy)[0:7]}，",end="\t")
        print('OK '+'='*20 if(np.max(np.array(accHistory)) == accuracy) else str(100*(accuracy - np.max(np.array(accHistory))))[0:5],end="\t")
    
    
    if(epoch>50):
        print(f"[{ str(np.abs(np.mean(lossHistory)-np.mean(lossHistory[0:epoch-50])))[0:4] }")
        if(np.abs(np.mean(lossHistory)-np.mean(lossHistory[0:epoch-50]))<0.05):
            if(np.abs(100*(accuracy - np.max(np.array(accHistory))))<5):
                print("已提前结束。")
                break
    else:
        print("")

epoch1: 21.9034	 acc_t[0.353]	 lr[0.001]	 acc_val: 0.36956，	OK ====================	
epoch2: 21.8804	 acc_t[0.373]	 lr[0.001]	 acc_val: 0.36956，	OK ====================	
epoch3: 21.8284	 acc_t[0.386]	 lr[0.001]	 acc_val: 0.36956，	OK ====================	
epoch4: 21.9113	 acc_t[0.369]	 lr[0.001]	 acc_val: 0.40217，	OK ====================	
epoch5: 21.8455	 acc_t[0.391]	 lr[0.001]	 acc_val: 0.38768，	-1.44	
epoch6: 21.7981	 acc_t[0.389]	 lr[0.001]	 acc_val: 0.40942，	OK ====================	
epoch7: 21.7968	 acc_t[0.395]	 lr[0.001]	 acc_val: 0.42028，	OK ====================	
epoch8: 21.6763	 acc_t[0.404]	 lr[0.001]	 acc_val: 0.39130，	-2.89	
epoch9: 21.7045	 acc_t[0.401]	 lr[0.001]	 acc_val: 0.39855，	-2.17	
epoch10: 21.6390	 acc_t[0.398]	 lr[0.001]	 acc_val: 0.39492，	-2.53	
epoch11: 21.6498	 acc_t[0.394]	 lr[0.001]	 acc_val: 0.39492，	-2.53	
epoch12: 21.4875	 acc_t[0.403]	 lr[0.001]	 acc_val: 0.39855，	-2.17	
epoch13: 21.6122	 acc_t[0.394]	 lr[0.001]	 acc_val: 0.40217，	-1.81	
epoch14: 21.5174	

epoch115: 20.4862	 acc_t[0.451]	 lr[0.001]	 acc_val: 0.44927，	-0.36	[0.31
epoch116: 20.0515	 acc_t[0.471]	 lr[0.001]	 acc_val: 0.44202，	-1.08	[0.31
epoch117: 20.1656	 acc_t[0.471]	 lr[0.001]	 acc_val: 0.43840，	-1.44	[0.32
epoch118: 20.2241	 acc_t[0.475]	 lr[0.001]	 acc_val: 0.44565，	-0.72	[0.32
epoch119: 20.1221	 acc_t[0.463]	 lr[0.001]	 acc_val: 0.48550，	OK ====================	[0.32
epoch120: 19.8984	 acc_t[0.471]	 lr[0.001]	 acc_val: 0.47826，	-0.72	[0.32
epoch121: 19.9869	 acc_t[0.485]	 lr[0.001]	 acc_val: 0.44565，	-3.98	[0.32
epoch122: 19.9867	 acc_t[0.479]	 lr[0.001]	 acc_val: 0.47101，	-1.44	[0.33
epoch123: 19.9165	 acc_t[0.481]	 lr[0.001]	 acc_val: 0.47826，	-0.72	[0.33
epoch124: 19.9830	 acc_t[0.472]	 lr[0.001]	 acc_val: 0.46376，	-2.17	[0.33
epoch125: 20.0065	 acc_t[0.487]	 lr[0.001]	 acc_val: 0.47463，	-1.08	[0.34
epoch126: 19.9437	 acc_t[0.472]	 lr[0.001]	 acc_val: 0.44565，	-3.98	[0.34
epoch127: 20.0940	 acc_t[0.464]	 lr[0.001]	 acc_val: 0.46014，	-2.53	[0.34
epoch128: 19.9698	 a

epoch219: 18.5651	 acc_t[0.530]	 lr[0.0009699999999999999]	 acc_val: 0.51449，	-2.89	[0.37
epoch220: 18.5198	 acc_t[0.534]	 lr[0.0009699999999999999]	 acc_val: 0.50362，	-3.98	[0.37
epoch221: 18.4219	 acc_t[0.533]	 lr[0.0009699999999999999]	 acc_val: 0.52173，	-2.17	[0.38
epoch222: 18.7175	 acc_t[0.527]	 lr[0.0009699999999999999]	 acc_val: 0.50724，	-3.62	[0.38
epoch223: 18.8186	 acc_t[0.522]	 lr[0.0009699999999999999]	 acc_val: 0.51449，	-2.89	[0.38
epoch224: 18.7517	 acc_t[0.526]	 lr[0.0009699999999999999]	 acc_val: 0.50724，	-3.62	[0.38
epoch225: 18.4190	 acc_t[0.535]	 lr[0.0009699999999999999]	 acc_val: 0.51449，	-2.89	[0.38
epoch226: 18.1012	 acc_t[0.539]	 lr[0.0009699999999999999]	 acc_val: 0.52536，	-1.81	[0.38
epoch227: 18.3327	 acc_t[0.538]	 lr[0.0009699999999999999]	 acc_val: 0.50362，	-3.98	[0.38
epoch228: 18.4200	 acc_t[0.536]	 lr[0.0009699999999999999]	 acc_val: 0.50362，	-3.98	[0.38
epoch229: 18.3131	 acc_t[0.540]	 lr[0.0009699999999999999]	 acc_val: 0.51086，	-3.26	[0.38
epoch230: 

epoch309: 17.7271	 acc_t[0.549]	 lr[0.0009699999999999999]	 acc_val: 0.48188，	-8.69	[0.34
epoch310: 17.5224	 acc_t[0.562]	 lr[0.0009699999999999999]	 acc_val: 0.54710，	-2.17	[0.34
epoch311: 17.4239	 acc_t[0.561]	 lr[0.0009699999999999999]	 acc_val: 0.55072，	-1.81	[0.34
epoch312: 17.8591	 acc_t[0.557]	 lr[0.0009699999999999999]	 acc_val: 0.52536，	-4.34	[0.34
epoch313: 17.8015	 acc_t[0.553]	 lr[0.0009699999999999999]	 acc_val: 0.54347，	-2.53	[0.34
epoch314: 17.2574	 acc_t[0.572]	 lr[0.0009699999999999999]	 acc_val: 0.53260，	-3.62	[0.34
epoch315: 17.9712	 acc_t[0.551]	 lr[0.0009699999999999999]	 acc_val: 0.51449，	-5.43	[0.33
epoch316: 17.8765	 acc_t[0.557]	 lr[0.0009699999999999999]	 acc_val: 0.51811，	-5.07	[0.33
epoch317: 17.4337	 acc_t[0.556]	 lr[0.0009699999999999999]	 acc_val: 0.53985，	-2.89	[0.33
epoch318: 17.4594	 acc_t[0.552]	 lr[0.0009699999999999999]	 acc_val: 0.52898，	-3.98	[0.33
epoch319: 17.4225	 acc_t[0.562]	 lr[0.0009699999999999999]	 acc_val: 0.50362，	-6.52	[0.33
epoch320: 

epoch400: 16.8362	 acc_t[0.577]	 lr[0.0009408999999999999]	 acc_val: 0.50362，	-6.52	[0.30
epoch401: 16.9260	 acc_t[0.576]	 lr[0.0009408999999999999]	 acc_val: 0.5，	-6.88	[0.29
epoch402: 17.0775	 acc_t[0.578]	 lr[0.0009408999999999999]	 acc_val: 0.51811，	-5.07	[0.29
epoch403: 16.6822	 acc_t[0.578]	 lr[0.0009408999999999999]	 acc_val: 0.49275，	-7.60	[0.29
epoch404: 16.9354	 acc_t[0.567]	 lr[0.0009408999999999999]	 acc_val: 0.50362，	-6.52	[0.29
epoch405: 16.5489	 acc_t[0.587]	 lr[0.0009408999999999999]	 acc_val: 0.51449，	-5.43	[0.29
epoch406: 16.1713	 acc_t[0.586]	 lr[0.0009408999999999999]	 acc_val: 0.53260，	-3.62	[0.29
epoch407: 16.4712	 acc_t[0.588]	 lr[0.0009408999999999999]	 acc_val: 0.50362，	-6.52	[0.29
epoch408: 16.7780	 acc_t[0.593]	 lr[0.0009408999999999999]	 acc_val: 0.48913，	-7.97	[0.29
epoch409: 16.9556	 acc_t[0.580]	 lr[0.0009408999999999999]	 acc_val: 0.52173，	-4.71	[0.29
epoch410: 16.1376	 acc_t[0.598]	 lr[0.0009408999999999999]	 acc_val: 0.50362，	-6.52	[0.30
epoch411: 16.3

...

epoch1036: 10.2883	 acc_t[0.759]	 lr[0.0008587340256999998]	 acc_val: 0.5，	-8.33	[0.25
epoch1037: 10.4670	 acc_t[0.742]	 lr[0.0008587340256999998]	 acc_val: 0.51086，	-7.24	[0.25
epoch1038: 10.7335	 acc_t[0.746]	 lr[0.0008587340256999998]	 acc_val: 0.48913，	-9.42	[0.25
epoch1039: 10.5819	 acc_t[0.746]	 lr[0.0008587340256999998]	 acc_val: 0.46739，	-11.5	[0.25
epoch1040: 11.8689	 acc_t[0.710]	 lr[0.0008587340256999998]	 acc_val: 0.52898，	-5.43	[0.24
epoch1041: 11.1837	 acc_t[0.743]	 lr[0.0008587340256999998]	 acc_val: 0.5，	-8.33	[0.24
epoch1042: 11.6639	 acc_t[0.723]	 lr[0.0008587340256999998]	 acc_val: 0.50362，	-7.97	[0.24
epoch1043: 10.3453	 acc_t[0.747]	 lr[0.0008587340256999998]	 acc_val: 0.51449，	-6.88	[0.24
epoch1044: 10.2226	 acc_t[0.747]	 lr[0.0008587340256999998]	 acc_val: 0.48550，	-9.78	[0.24
epoch1045: 10.7447	 acc_t[0.735]	 lr[0.0008587340256999998]	 acc_val: 0.48188，	-10.1	[0.24
epoch1046: 11.5719	 acc_t[0.728]	 lr[0.0008587340256999998]	 acc_val: 0.51449，	-6.88	[0.24
epoch10

epoch1126: 9.89372	 acc_t[0.760]	 lr[0.0008587340256999998]	 acc_val: 0.52898，	-5.43	[0.22
epoch1127: 9.36092	 acc_t[0.770]	 lr[0.0008587340256999998]	 acc_val: 0.52898，	-5.43	[0.23
epoch1128: 9.49208	 acc_t[0.770]	 lr[0.0008587340256999998]	 acc_val: 0.53985，	-4.34	[0.23
epoch1129: 9.45369	 acc_t[0.777]	 lr[0.0008587340256999998]	 acc_val: 0.5，	-8.33	[0.23
epoch1130: 9.85524	 acc_t[0.753]	 lr[0.0008587340256999998]	 acc_val: 0.51449，	-6.88	[0.23
epoch1131: 10.4700	 acc_t[0.742]	 lr[0.0008587340256999998]	 acc_val: 0.54710，	-3.62	[0.23
epoch1132: 9.44685	 acc_t[0.777]	 lr[0.0008587340256999998]	 acc_val: 0.52536，	-5.79	[0.23
epoch1133: 9.31871	 acc_t[0.770]	 lr[0.0008587340256999998]	 acc_val: 0.54347，	-3.98	[0.23
epoch1134: 9.36085	 acc_t[0.765]	 lr[0.0008587340256999998]	 acc_val: 0.53260，	-5.07	[0.23
epoch1135: 9.01076	 acc_t[0.778]	 lr[0.0008587340256999998]	 acc_val: 0.52173，	-6.15	[0.23
epoch1136: 9.29648	 acc_t[0.772]	 lr[0.0008587340256999998]	 acc_val: 0.51449，	-6.88	[0.23
epo

epoch1216: 9.36316	 acc_t[0.764]	 lr[0.0008329720049289999]	 acc_val: 0.50362，	-7.97	[0.22
epoch1217: 10.4899	 acc_t[0.764]	 lr[0.0008329720049289999]	 acc_val: 0.52173，	-6.15	[0.22
epoch1218: 10.9470	 acc_t[0.739]	 lr[0.0008329720049289999]	 acc_val: 0.49637，	-8.69	[0.22
epoch1219: 9.83216	 acc_t[0.756]	 lr[0.0008329720049289999]	 acc_val: 0.51449，	-6.88	[0.22
epoch1220: 8.91317	 acc_t[0.785]	 lr[0.0008329720049289999]	 acc_val: 0.52173，	-6.15	[0.22
epoch1221: 9.14488	 acc_t[0.776]	 lr[0.0008329720049289999]	 acc_val: 0.48913，	-9.42	[0.22
epoch1222: 8.97179	 acc_t[0.791]	 lr[0.0008329720049289999]	 acc_val: 0.46739，	-11.5	[0.22
epoch1223: 10.4500	 acc_t[0.762]	 lr[0.0008329720049289999]	 acc_val: 0.49275，	-9.05	[0.22
epoch1224: 9.78002	 acc_t[0.752]	 lr[0.0008329720049289999]	 acc_val: 0.50724，	-7.60	[0.22
epoch1225: 8.74254	 acc_t[0.787]	 lr[0.0008329720049289999]	 acc_val: 0.50724，	-7.60	[0.22
epoch1226: 9.22702	 acc_t[0.778]	 lr[0.0008329720049289999]	 acc_val: 0.44202，	-14.1	[0.22

epoch1306: 9.08747	 acc_t[0.778]	 lr[0.0008329720049289999]	 acc_val: 0.52173，	-6.15	[0.20
epoch1307: 8.87801	 acc_t[0.788]	 lr[0.0008329720049289999]	 acc_val: 0.51086，	-7.24	[0.20
epoch1308: 9.00025	 acc_t[0.775]	 lr[0.0008329720049289999]	 acc_val: 0.51086，	-7.24	[0.20
epoch1309: 8.98215	 acc_t[0.776]	 lr[0.0008329720049289999]	 acc_val: 0.49637，	-8.69	[0.20
epoch1310: 8.95433	 acc_t[0.789]	 lr[0.0008329720049289999]	 acc_val: 0.47826，	-10.5	[0.20
epoch1311: 8.82748	 acc_t[0.783]	 lr[0.0008329720049289999]	 acc_val: 0.51449，	-6.88	[0.20
epoch1312: 9.25913	 acc_t[0.782]	 lr[0.0008329720049289999]	 acc_val: 0.51811，	-6.52	[0.20
epoch1313: 8.92152	 acc_t[0.773]	 lr[0.0008329720049289999]	 acc_val: 0.49275，	-9.05	[0.20
epoch1314: 8.58380	 acc_t[0.794]	 lr[0.0008329720049289999]	 acc_val: 0.5，	-8.33	[0.20
epoch1315: 8.53031	 acc_t[0.797]	 lr[0.0008329720049289999]	 acc_val: 0.51811，	-6.52	[0.20
epoch1316: 8.62408	 acc_t[0.789]	 lr[0.0008329720049289999]	 acc_val: 0.51086，	-7.24	[0.20
epo

epoch1396: 8.98141	 acc_t[0.784]	 lr[0.0008329720049289999]	 acc_val: 0.48550，	-9.78	[0.20
epoch1397: 9.14880	 acc_t[0.780]	 lr[0.0008329720049289999]	 acc_val: 0.50362，	-7.97	[0.20
epoch1398: 11.1048	 acc_t[0.751]	 lr[0.0008329720049289999]	 acc_val: 0.49637，	-8.69	[0.20
epoch1399: 10.1950	 acc_t[0.766]	 lr[0.0008329720049289999]	 acc_val: 0.55072，	-3.26	[0.20
epoch1400: 8.92048	 acc_t[0.787]	 lr[0.0008079828447811299]	 acc_val: 0.51449，	-6.88	[0.20
epoch1401: 8.65417	 acc_t[0.783]	 lr[0.0008079828447811299]	 acc_val: 0.51449，	-6.88	[0.20
epoch1402: 8.73463	 acc_t[0.783]	 lr[0.0008079828447811299]	 acc_val: 0.48188，	-10.1	[0.20
epoch1403: 9.16814	 acc_t[0.787]	 lr[0.0008079828447811299]	 acc_val: 0.47101，	-11.2	[0.20
epoch1404: 8.97125	 acc_t[0.774]	 lr[0.0008079828447811299]	 acc_val: 0.48188，	-10.1	[0.20
epoch1405: 8.86736	 acc_t[0.789]	 lr[0.0008079828447811299]	 acc_val: 0.50724，	-7.60	[0.20
epoch1406: 7.91395	 acc_t[0.803]	 lr[0.0008079828447811299]	 acc_val: 0.48188，	-10.1	[0.20

In [ ]:
# 0 43
# 60 42



x = range(0,epochs) # x轴坐标值113  0.00009 -> 81
plt.plot(trainAccHis,c = 'r') # 参数c为color简写，表示颜色,r为red即红色

In [ ]:
x = range(0,epochs) # x轴坐标值
plt.plot(lossHistory,c = 'r') # 参数c为color简写，表示颜色,r为red即红色

In [ ]:

x = range(0,epochs) # x轴坐标值
plt.plot(accHistory,c = 'r') # 参数c为color简写，表示颜色,r为red即红色


In [ ]:
x = range(0,epochs) # x轴坐标值
plt.plot(accAverHis,c = 'r') # 参数c为color简写，表示颜色,r为red即红色



In [ ]:

with open("temp.pkl", 'wb') as file:
    pickle.dump(dict({"loss":lossHistory,"acc":accHistory,"trainAccHis":trainAccHis}), file)
# with open("model_LSTM_EDA300_10(10).pkl", 'wb') as file:
#     pickle.dump(dict({"loss":lossHistory,"acc":accHistory,"trainAccHis":trainAccHis}), file)

In [ ]:
model.eval()
with torch.no_grad():
    all_outputs = []
    all_targets = []
    for batch_data, batch_targets in dataLoader_valid:
        outputs = model(batch_data.to(device))
        _, predicted = torch.max(outputs.data, 1)
#         print(batch_data)
#         if()
        print(outputs.data)
        all_outputs.append(predicted.cpu().numpy())
        all_targets.append(batch_targets.cpu().numpy())

    all_outputs = np.concatenate(all_outputs)
    print(all_outputs)
    all_targets = np.concatenate(all_targets)
    print(all_targets)
    accuracy = accuracy_score(all_targets, all_outputs)

print(f" Accuracy: {accuracy}")

In [ ]:
modelB.eval()
dataLoader_valid = DataLoader(TensorDataset(dataX[rnm[-1][1]],dataY[rnm[-1][1]]), batch_size=64, shuffle=False,num_workers=4)
with torch.no_grad():
    all_outputs = []
    all_targets = []
    for batch_data, batch_targets in dataLoader_valid:
        outputs = modelB(batch_data.to(device))
        _, predicted = torch.max(outputs.data, 1)
#         print(batch_data)
#         if()
        print(outputs.data)
        all_outputs.append(predicted.cpu().numpy())
        all_targets.append(batch_targets.cpu().numpy())

    all_outputs = np.concatenate(all_outputs)
    print(all_outputs)
    all_targets = np.concatenate(all_targets)
    print(all_targets)
    accuracy = accuracy_score(all_targets, all_outputs)

print(f" Accuracy: {accuracy}")

In [ ]:
torch.save({'model': model.state_dict()}, 'model_LSTM_EDA300_10(10).pth')

In [ ]:
state_dict = torch.load('model_CNN_EDA_10_new.pth')
modelB = Model()
modelB.load_state_dict(state_dict['model'])
modelB = modelB.to(device)
modelB.eval()
with torch.no_grad():
    all_outputs = []
    all_targets = []
    for batch_data, batch_targets in dataLoader_valid:
        outputs = modelB(batch_data.to(device))
        _, predicted = torch.max(outputs.data, 1)
        all_outputs.append(predicted.cpu().numpy())
        all_targets.append(batch_targets.cpu().numpy())

    all_outputs = np.concatenate(all_outputs)
    all_targets = np.concatenate(all_targets)
    accuracy = accuracy_score(all_targets, all_outputs)
   
    print(f"验证集{1+1}: {accuracy}，",end="")

In [ ]:
#  EPOCH =  2000 LR = 0.01 - 0.0005
epochs = 1000
optimizer = torch.optim.Adam(modelB.parameters(), lr=0.0003)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(epochs):
    modelB.train()
    LOSS = 0
    for batch_data, batch_targets in dataLoader_train:
        optimizer.zero_grad()
        outputs = modelB(batch_data.to(device))
        loss = criterion(outputs, batch_targets.to(device))
        loss.backward()
#         print(outputs)
#         print(batch_targets)
        LOSS += loss.detach().cpu().numpy()
        optimizer.step()
    print(f"epoch{epoch+1}: {LOSS}")

In [ ]:
torch.save({'model': modelB.state_dict()}, '3000epoch.pth')

In [ ]:
a=[[0 for x in range(column)] for y in range(row)]
some_string = ''
for i in range (0, 8):
    for j in range (0, 7):
        some_string +="a[{}][{}]= {}".format(i,j,a[i][j])
print(some_string)